In [1]:
import pandas as pd

In [37]:
class ProductionOrderScheduler:
    SMALL_ORDER_TRRESHOLD = 4  # Threshold for small orders in quantity
    TRIPLE_GLAZED_PANES = ['9', '9C']
    URGENT_ORDERS_RECEIVERS_2_pm = ['2101/Polska/C', '3301/Węgry/C']
    SAP_NUMBERS_FOR_FIRST_AND_LAST_POSITIONS = ['808965', '808966']
    TWO_SHIFTS_TRESHOLD = 180  # Threshold for two shifts in quantity
    MIDDLE_POINT_PROPORTION = 0.55  # Proportion of the sum of windows per shift to determine the middle point
    ADDITIONAL_MILLING_WIDTHS = [1340]  # Widths that require additional milling operations
    ADDITIONAL_MILLING_VARIANTS = ['EXL', 'PRO']  # Variants that require additional milling operations
    MILLED_WINDOWS_MAX_SEQUENCE = 14  # Maximum sequence of milled windows in the production plan [pcs]
    MILLED_WINDOWS_SEQUENCE_SEPARATION = 6  # Separation between milled windows in the production plan [pcs]

    def __init__(self):
        """
        Initialize the scheduler with production data
        """
        self.current_record_num = 1

        self.total_sum_of_windows = 0
        self.middle_point = 0 # Middle point of the production plan - 'kreska', frozen part of the plan

        self.total_num_of_small_orders = 0
        self.total_num_of_first_and_last_positions_orders = 0
        
        self.sum_of_triple_glazed = 0
        self.sum_of_milled_orders = 0
        self.sum_of_golden_oak_triple = 0
        self.sum_of_pine_triple = 0
        self.sum_of_golden_oak_double = 0
        self.sum_of_pine_double = 0
        self.sum_golden_oak_triple_urgent = 0
        self.sum_pine_triple_urgent = 0

        self.unique_widths = set()  # Set to store unique widths of windows
        self.last_width_index = 0  # Index of the last width in the unique widths list

        self.colors_list = ['K', 'G']
        self.color_before_middle_point = None  # Color of the windows with triple panes before the middle point

        # milled windows sequence parameters
        self.milled_windows_max_sequence = self.MILLED_WINDOWS_MAX_SEQUENCE
        self.milled_windows_sequence_separation = self.MILLED_WINDOWS_SEQUENCE_SEPARATION

        # variables defining last scheduled order
        self.last_order_width = None
        self.last_order_height = None
        self.last_order_type = None
        self.last_order_glass = None
        self.last_order_color = None
        self.last_order_is_milled = None
        self.last_order_is_triple = None
        
        # load the data into a DataFrame
        self.load_production_plan()
        
        # add new columns to the DataFrame based on various conditions
        self.is_small_order()
        self.is_triple()
        self.is_urgent_till_2_pm()
        self.is_urgent_till_6_pm()
        self.is_material_available()
        self.add_scheduling_columns()
        self.is_milled_window()

        # Calculate some statistics
        self.calculate_triple_glazed()
        self.calculate_milled()
        self.calculate_golden_oak_and_pine()
        self.calculate_total_sum_of_windows()
        self.count_small_orders()
        self.count_first_and_last_positions_orders()
        self.calculate_middle_point()
        self.get_unique_widths()

        self.define_milled_windows_sequence_parameters()
        self.define_color_before_middle_point()
        
        self.scheduled_orders = []

    def load_production_plan(self):
        # get data from clipboard
        self.production_plan_df = pd.read_clipboard(sep='\t', header=0, index_col=0, dtype={'sap_nr': str})
        # sort the DataFrame by 'glass_type' and 'width'
        self.production_plan_df.sort_values(by=['profile_color', 'variant', 'glass_type', 'width'], inplace=True)

    def is_small_order(self):
        """
        Check if the order is small based on its size
        """
        self.production_plan_df['is_small'] = self.production_plan_df['quantity'] < self.SMALL_ORDER_TRRESHOLD

    def is_triple(self):
        """
        Check if the order is a triple-glazed window order
        """
        self.production_plan_df['is_triple'] = self.production_plan_df['glass_type'].isin(self.TRIPLE_GLAZED_PANES)

    def calculate_triple_glazed(self):
        """
        Calculate the number of triple-glazed windows in the production plan
        """
        self.sum_of_triple_glazed = self.production_plan_df[self.production_plan_df['is_triple']]['quantity'].sum()

    def calculate_milled(self):
        """
        Calculate the number of EXL and PRO orders in the production plan
        """
        self.sum_of_milled_orders = self.production_plan_df[self.production_plan_df['is_milled']]['quantity'].sum()

    def calculate_golden_oak_and_pine(self):
        """
        Calculate the number of Golden Oak and Pine orders in the production plan
        """
        self.sum_of_golden_oak_triple = self.production_plan_df[(self.production_plan_df['profile_color'] == 'G') & (self.production_plan_df['is_triple'])]['quantity'].sum()
        self.sum_of_pine_triple = self.production_plan_df[(self.production_plan_df['profile_color'] == 'K') & (self.production_plan_df['is_triple'])]['quantity'].sum()
        self.sum_of_golden_oak_double = self.production_plan_df[(self.production_plan_df['profile_color'] == 'G') & (~self.production_plan_df['is_triple'])]['quantity'].sum()
        self.sum_of_pine_double = self.production_plan_df[(self.production_plan_df['profile_color'] == 'K') & (~self.production_plan_df['is_triple'])]['quantity'].sum()
        self.sum_golden_oak_triple_urgent = self.production_plan_df[(self.production_plan_df['profile_color'] == 'G') & 
                                                                  (self.production_plan_df['is_triple']) & 
                                                                  (self.production_plan_df['goods_receiver']).str.endswith('/C')]['quantity'].sum()
        self.sum_pine_triple_urgent = self.production_plan_df[(self.production_plan_df['profile_color'] == 'K') & 
                                                                  (self.production_plan_df['is_triple']) & 
                                                                  (self.production_plan_df['goods_receiver']).str.endswith('/C')]['quantity'].sum()

    def calculate_total_sum_of_windows(self):
        """
        Calculate the total sum of windows in the production plan
        """
        self.total_sum_of_windows = self.production_plan_df['quantity'].sum()

    def count_small_orders(self):
        """
        Count the number of small orders in the production plan
        """
        self.total_num_of_small_orders = self.production_plan_df[self.production_plan_df['is_small']].shape[0]

    def count_first_and_last_positions_orders(self):
        """
        Count the number of orders that can be either first or last positions in the production plan
        """
        self.total_num_of_first_and_last_positions_orders = self.production_plan_df[
            self.production_plan_df['sap_nr'].isin(self.SAP_NUMBERS_FOR_FIRST_AND_LAST_POSITIONS)
        ].shape[0]
        
    def is_urgent_till_2_pm(self):
        """
        Check if the order is urgent and needs to be completed by 2 PM
        """
        self.production_plan_df['is_urgent_till_2_pm'] = self.production_plan_df['goods_receiver'].isin(self.URGENT_ORDERS_RECEIVERS_2_pm)

    def is_urgent_till_6_pm(self):
        """
        Check if the order is urgent and needs to be completed by 6 PM
        """
        self.production_plan_df['is_urgent_till_6_pm'] = self.production_plan_df['goods_receiver'].str.endswith('/C', na=False) & ~self.production_plan_df['goods_receiver'].isin(self.URGENT_ORDERS_RECEIVERS_2_pm)

    def is_material_available(self):
        """
        Check if the material is available for given production order
        """
        self.production_plan_df['is_material_available'] = self.production_plan_df['system_status'].str.startswith('ZWOL')

    def is_milled_window(self):
        """
        Check if the window has additional milling operations
        """
        self.production_plan_df['is_milled'] = self.production_plan_df.apply(lambda row: row['variant'] in self.ADDITIONAL_MILLING_VARIANTS or row['width'] in self.ADDITIONAL_MILLING_WIDTHS, axis=1)

    def get_unique_widths(self):
        """
        Get unique widths of windows in the production plan
        """
        self.unique_widths = list(set(self.production_plan_df['width'].unique()))
        self.unique_widths.sort()  # Sort the unique widths for better readability

    def calculate_middle_point(self):
        """
        Calculate the middle point of the production plan based on the quantity of windows
        Middle point is so called "kreska" which is used to determine the point till which the plan if 'frozen'
        """
        if self.total_sum_of_windows >= self.TWO_SHIFTS_TRESHOLD:
            # two shifts production
            self.middle_point = int(self.MIDDLE_POINT_PROPORTION * (self.total_sum_of_windows // 2))
        else:
            # one shift production
            self.middle_point = int(self.MIDDLE_POINT_PROPORTION * self.total_sum_of_windows)

    def add_scheduling_columns(self):
        """
        Add a column to the DataFrame indicating the scheduling position of each order
        False means that the order is not scheduled yet
        1 means that the order is scheduled for the first position and so on
        """
        self.production_plan_df['scheduling_position'] = None  # Initialize with None
        self.production_plan_df['is_scheduled'] = False  # Initialize with False

    def define_color_before_middle_point(self):
        """
        Define colors of windows with triple panes before the middle point of the production plan
        G or K
        """
        if self.sum_of_golden_oak_triple == 0 and self.sum_of_pine_triple == 0:
            return  # No triple glazed windows to define color
        if self.sum_golden_oak_triple_urgent > self.sum_pine_triple_urgent:
            self.color_before_middle_point = 'G'  # Golden Oak
        elif self.sum_golden_oak_triple_urgent < self.sum_pine_triple_urgent:
            self.color_before_middle_point = 'K'
        else:
            if self.sum_of_golden_oak_triple >= self.sum_of_pine_triple:
                self.color_before_middle_point = 'G'
            else:
                self.color_before_middle_point = 'K'

    def define_milled_windows_sequence_parameters(self):
        """
        Define parameters for milled windows sequence in the production plan
        """
        if self.sum_of_milled_orders / (self.total_sum_of_windows - self.sum_of_triple_glazed - self.sum_of_milled_orders) > self.milled_windows_max_sequence / self.milled_windows_sequence_separation:
            # If the proportion of milled orders is too high, adjust the parameters
            desired_proportion = self.MILLED_WINDOWS_MAX_SEQUENCE / self.MILLED_WINDOWS_SEQUENCE_SEPARATION
            current_proportion = self.sum_of_milled_orders / (self.total_sum_of_windows - self.sum_of_triple_glazed - self.sum_of_milled_orders)
            if current_proportion > desired_proportion:
                self.milled_windows_max_sequence = int(self.MILLED_WINDOWS_MAX_SEQUENCE * (current_proportion / desired_proportion))
        else:
            return

    def schedule_one_position(self, df_row):
        """
        Schedule one position in the production plan
        """
        self.production_plan_df.at[df_row.Index, 'scheduling_position'] = self.current_record_num
        self.production_plan_df.at[df_row.Index, 'is_scheduled'] = True
        self.current_record_num += 1

        # Update last scheduled order details
        self.last_order_width = df_row.width
        self.last_order_height = df_row.height
        self.last_order_is_milled = df_row.is_milled
        self.last_order_is_triple = df_row.is_triple
        self.last_order_type = df_row.window_type
        self.last_order_glass = df_row.glass_type
        self.last_order_color = df_row.profile_color
        self.last_width_index = self.unique_widths.index(df_row.width)
        
    def start_the_production_plan(self):
        """Select first positions to production plan R79 7/11 and 7/14"""
        num_of_orders_to_plan = self.total_num_of_first_and_last_positions_orders // 2
        counter = 0

        for row in self.production_plan_df.itertuples():
            if row.sap_nr in self.SAP_NUMBERS_FOR_FIRST_AND_LAST_POSITIONS and not row.is_scheduled:
                self.schedule_one_position(row)
                counter += 1
                if counter >= num_of_orders_to_plan:
                    break

    def main_scheduling_function(self):
        """
        Schedule production orders based on the production plan and defined conditions
        """
        # Here you can implement the logic to schedule the production orders
        # For now, we will just return the DataFrame with scheduled orders
        self.start_the_production_plan()
        
    

In [42]:
scheduler = ProductionOrderScheduler()
print("sum of triple glazed:", scheduler.sum_of_triple_glazed)
print("sum of EXL and PRO orders:", scheduler.sum_of_milled_orders)
print("sum of Golden Oak triple orders:", scheduler.sum_of_golden_oak_triple)
print("sum of Pine triple orders:", scheduler.sum_of_pine_triple)
print("sum of Golden Oak double orders:", scheduler.sum_of_golden_oak_double)
print("sum of Pine double orders:", scheduler.sum_of_pine_double)
print("Sum of Golden Oak triple urgent orders:", scheduler.sum_golden_oak_triple_urgent)
print("Sum of Pine triple urgent orders:", scheduler.sum_pine_triple_urgent)
print("Color before middle point:", scheduler.color_before_middle_point)
print("\n")
print("total sum of windows:", scheduler.total_sum_of_windows)
print("total number of small orders:", scheduler.total_num_of_small_orders)
print("total number of first and last positions orders:", scheduler.total_num_of_first_and_last_positions_orders)
print("middle point of the production plan:", scheduler.middle_point)
print("Unique widths of windows:", scheduler.unique_widths)
print("\n")
print("Milled windows max sequence:", scheduler.milled_windows_max_sequence)
print("Milled windows sequence separation:", scheduler.milled_windows_sequence_separation)
print("\n")
scheduler.main_scheduling_function()
print("Last width index:", scheduler.last_width_index)

scheduler.production_plan_df

sum of triple glazed: 154
sum of EXL and PRO orders: 90
sum of Golden Oak triple orders: 8
sum of Pine triple orders: 0
sum of Golden Oak double orders: 0
sum of Pine double orders: 0
Sum of Golden Oak triple urgent orders: 4
Sum of Pine triple urgent orders: 0
Color before middle point: G


total sum of windows: 294
total number of small orders: 20
total number of first and last positions orders: 3
middle point of the production plan: 80
Unique widths of windows: [np.int64(540), np.int64(650), np.int64(740), np.int64(780), np.int64(1140), np.int64(1340)]


Milled windows max sequence: 14
Milled windows sequence separation: 6


Last width index: 2


,goods_receiver,production_order_number,sap_nr,product_name,quantity,system_status,glass_type,profile_color,width,height,variant,window_type,is_small,is_triple,is_urgent_till_2_pm,is_urgent_till_6_pm,is_material_available,scheduling_position,is_scheduled,is_milled
record_number,,,,,,,,,,,,,,,,,,,,
10,2101/Polska/C,115774561,990510,R79__074/160_G800L5,2,OTW FMAT KWS BŁKL ODZT PRRO,9,G,740,1600,EXL,R7,True,True,True,False,False,None,False,True
46,2101/Polska/C,115775673,990510,R79__074/160_G800L5,2,OTW FMAT KWS BŁKL ODZT PRRO,9,G,740,1600,EXL,R7,True,True,True,False,False,None,False,True
30,3701/Czechy/Z,115780770,838284,R79_ 074/098 G200,2,ZWOL KWS MTZT PRRO,9,G,740,980,HAN,R7,True,True,False,False,True,None,False,False
37,3701/Czechy/Z,115781882,838284,R79_ 074/098 G200,2,ZWOL KWS MTZT PRRO,9,G,740,980,HAN,R7,True,True,False,False,True,None,False,False
18,2101/Polska,115778976,496514,R45_ 054/098 K100L5,7,ZWOL KWS MTZT PRRO,5,W,540,980,EXL,R4,False,False,False,False,True,None,False,True
25,0301/Niemcy,115780363,496513,R45_ 054/078 K100L5,4,ZWOL KWS MTZT PRRO,5,W,540,780,EXL,R4,False,False,False,False,True,None,False,True
32,0301/Niemcy,115781475,496513,R45_ 054/078 K100L5,14,ZWOL KWS MTZT PRRO,5,W,540,780,EXL,R4,False,False,False,False,True,None,False,True
15,0301/Niemcy,115777761,744932,R48_ 054/078 K100L5,7,OTW FMAT KWS ODZT PRRO,8,W,540,780,EXL,R4,False,False,False,False,False,None,False,True
17,0301/Niemcy/Z,115778872,990510,R78__078/098_K100L5,1,ZWOL KWS MTZT PRRO,8,W,780,980,EXL,R7,True,False,False,False,True,None,False,True


In [28]:
def ping_pong_iter(lst, start_index=0, steps=10):
    n = len(lst)
    if n == 0 or steps <= 0:
        return

    index = start_index
    direction = 1  # 1 = forward, -1 = backward

    for _ in range(steps):
        yield lst[index]

        if index == n - 1:
            direction = -1
        elif index == 0:
            direction = 1

        index += direction

for width in ping_pong_iter(scheduler.unique_widths, start_index=2, steps=len(scheduler.unique_widths)*2-3):
    print(width)

740
780
1140
1340
1140
780
740
650
540


In [29]:
scheduler.unique_widths

[np.int64(540),
 np.int64(650),
 np.int64(740),
 np.int64(780),
 np.int64(1140),
 np.int64(1340)]